In [ ]:
# Imports you may need
from dataloader import *
from finance import *
import pandas as pd
from tqdm import tqdm

from finance import stock, compare
from quotebankexploration import *
from wikipedia import *

import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import yfinance as yf

import plotly
import plotly.figure_factory as ff
import plotly.tools as tls
import plotly.io as pio
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
from scipy import signal


from ipywidgets import interactive, HBox, VBox, Checkbox

%load_ext autoreload
%autoreload 2

tqdm.pandas()

In [ ]:
# Get some data frames
wiki_data = concat_wiki_files()
filtered_quotes = get_filtered_quotes()
speakers_id = pd.DataFrame(get_speakers_ids(filtered_quotes))
speakers_labels = get_speakers_labels()

In [ ]:
quotes_test = filtered_quotes.sample(100).reset_index(drop = True)
id_test = pd.DataFrame(get_speakers_ids(quotes_test))
labels_test = find_labels(id_test, wiki_data)

In [ ]:
for year in range(2015, 2021):
    labels_test[str(year)] = labels_test.label.apply(lambda label: get_pageviews_per_year(label, year))
    print(year)

In [ ]:
quotes_test['year'] = quotes_test.date.apply(lambda date: date.year)

In [ ]:
yolo = quotes_test.set_index('speaker').join(labels_test.set_index('speaker'), lsuffix="_left", rsuffix="_right").reset_index()
display(yolo)

In [ ]:
def scoring(quotes):
    try:
        score = quotes[str(quotes['year'])]
    except:
        score = None
    return score

shit = yolo.copy()
shit['score'] = yolo.apply(scoring, axis = 1)
shit = shit.drop(['2015', '2016', '2017', '2018', '2019', '2020', 'year'], axis = 1)
display(shit)

# Feelings

***

In [ ]:
analyzer = SentimentIntensityAnalyzer()
# determine the sentiment of a quote in a corpus (positive, negative or neutral)
def sentiment(quote) : 
    vs = analyzer.polarity_scores(quote)['compound']
    if (vs >= 1) :
        return 1
    if (vs <= - 1) :
        return -1
    else : return vs

fuck = shit.copy()
fuck['sentiment'] = fuck['quotation'].apply(sentiment) 
df_sent = fuck.groupby(['sentiment']).sum().reset_index()
display(fuck)

# Get back 2015 / 2016

***

In [ ]:
prout_2015 = pd.read_pickle('data/wiki_speaker_attributes/speakers_pageviews_2015')
prout_2016 = pd.read_pickle('data/wiki_speaker_attributes/speakers_pageviews_2016')
prout_2017 = pd.read_pickle('data/wiki_speaker_attributes/speakers_pageviews_2017')
prout_2018 = pd.read_pickle('data/wiki_speaker_attributes/speakers_pageviews_2018')
prout_2019 = pd.read_pickle('data/wiki_speaker_attributes/speakers_pageviews_2019')
prout_2020 = pd.read_pickle('data/wiki_speaker_attributes/speakers_pageviews_2020')
prout = prout_2015.copy()
prout['2016'] = prout_2016['2016']
prout['2017'] = prout_2017['2017']
prout['2018'] = prout_2018['2018']
prout['2019'] = prout_2019['2019']
prout['2020'] = prout_2020['2020']

In [ ]:
quotes_prout = filtered_quotes.copy()
quotes_prout['year'] = quotes_prout.date.apply(lambda date: date.year)
df = quotes_prout.set_index('speaker').join(prout.set_index('speaker'), lsuffix="_left", rsuffix="_right").reset_index()

In [ ]:
df_ = df[df.year >= 2015].copy()
df_['absolute_score'] = df.apply(scoring, axis = 1)
df_ = df_.drop(['2015', '2016', '2017', '2018', '2019', '2020', 'year'], axis = 1)

In [ ]:
df_final = df_.copy()
df_final['sentiment'] = df_final['quotation'].progress_apply(sentiment)

In [ ]:
def func_pos(row):
    score = 0
    if row.sentiment > 0:
        score = row.absolute_score
    return score

def func_neg(row):
    score = 0
    if row.sentiment < 0:
        score = -row.absolute_score
    return score

df_final_ = df_final.copy()
df_final_['score_pos'] = df_final_.progress_apply(func_pos, axis = 1)
df_final_['score_neg'] = df_final_.progress_apply(func_neg, axis = 1).abs()

MIN = min(df_final_['score_pos'].min(), df_final_['score_neg'].min())
MAX = max(df_final_['score_pos'].max(), df_final_['score_neg'].max())

df_final_['score_pos'] = (df_final_['score_pos'] - MIN) / (MAX - MIN)
df_final_['score_neg'] = -(df_final_['score_neg'] - MIN) / (MAX - MIN)

df_final_ = df_final_[['date', 'score_pos', 'score_neg']].dropna().groupby(['date']).sum()
df_final_ = df_final_.reset_index(drop = False)
df_final_ = df_final_.sort_values(by="date")

df_final_ = pd.DataFrame(df_final_.groupby(df_final_.date.dt.date).sum())

window = 7

steve = df_final_.copy()
steve['pos'] = steve.score_pos.rolling(window = window, min_periods = 1).mean()
steve['neg'] = steve.score_neg.rolling(window = window, min_periods = 1).mean()
# steve['pos'] = steve.score_pos.rolling(window=window, min_periods = 1, center = True, win_type='gaussian').mean(std=3)

# steve['pos'][:window] = steve.score_pos[:window].mean()
# steve['neg'][:window] = steve.score_neg[:window].mean()

df_final_ = steve

df_final_['score'] = df_final_.score_pos + df_final_.score_neg

df_final_ = df_final_.reset_index(drop = False)

display(df_final_)

In [ ]:
df_final_.tail(1).date.values

In [ ]:
stock_name = "AAPL"
year_start = 2015
year_end = 2020
stock = yf.download(stock_name, start=f'{year_start}-01-01', end=f'{year_end}-12-31', progress = False)
stock.reset_index(inplace=True)
date_max = '2020-04-16'
stock = stock[stock.Date <= date_max]

In [ ]:
stock_analysis = stock.copy()
stock_analysis['diff_'] = stock_analysis.Close - stock_analysis.Open

window = 30

stock_analysis['diff_mean'] = stock_analysis['diff_'].rolling(window = window, min_periods = 1).mean()

stock_analysis['stock_price'] = (stock_analysis.Open + stock_analysis.Low) / 2

display(stock_analysis)


In [ ]:
from scipy.stats import pearsonr

stock_analysis.Date = pd.to_datetime(stock_analysis.Date)
df_final_.date = pd.to_datetime(df_final_.date)
intersection_stock = pd.Index(stock_analysis.Date.isin(set(stock_analysis.Date).intersection(set(df_final_.date))))
intersection_quotes = pd.Index(df_final_.date.isin(set(stock_analysis.Date).intersection(set(df_final_.date))))

stock_analysis_keep = stock_analysis[intersection_stock]
df_final_keep = df_final_[intersection_quotes]

z = pearsonr(stock_analysis_keep.diff_ , df_final_keep.score) 
print(z)

In [ ]:
(0.004640995842596226, 0.8681492519121116)

In [ ]:
trace1 = go.Bar(
    x = df_final_.date,
    y = df_final_.score_pos,
    name = 'Positive',
    marker=dict(color='rgb(0,200,0)')
)

trace2 = go.Bar(
    x = df_final_.date,
    y = df_final_.score_neg,
    name = 'Negative',
    marker=dict(color='rgb(200,0,0)')
)

# ------------------------------------- #


trace1 = go.Scatter(
    x = df_final_.date,
    y = signal.savgol_filter(df_final_.score_pos,
                           53, # window size used for filtering
                           3), # order of fitted polynomial
    mode = 'lines',
    name = 'Positive',
    marker=dict(color='rgb(0,200,0)')
)

trace2 = go.Scatter(
    x = df_final_.date,
    y = signal.savgol_filter(df_final_.score_neg,
                           53, # window size used for filtering
                           3), # order of fitted polynomial
    mode = 'lines',
    name = 'Negative',
    marker=dict(color='rgb(200,0,0)')
)


# -------------------------------------- #

trace1 = go.Scatter(
    x = df_final_.date,
    y = df_final_.pos,
    mode = 'lines',
    name = 'Positive',
    marker=dict(color='rgb(30,50,130)')
)

trace2 = go.Scatter(
    x = df_final_.date,
    y = df_final_.neg,
    mode = 'lines',
    name = 'Negative',
    marker=dict(color='rgb(150,37,30)')
)

In [ ]:
trace1 = go.Scatter(
    x = df_final_.date,
    y = df_final_.pos,
    mode = 'lines',
    name = 'Positive',
    marker=dict(color='rgb(30,50,130)')
)

trace2 = go.Scatter(
    x = df_final_.date,
    y = df_final_.neg,
    mode = 'lines',
    name = 'Negative',
    marker=dict(color='rgb(150,37,30)')
)

trace3 = go.Scatter(
    x = stock_analysis.Date,
    y = stock_analysis.stock_price,
    mode = 'lines',
    name = 'Stock',
    # marker=dict(color='rgb(250,125,62)'),
    marker=dict(color='rgb(25,125,35)'),
    opacity = 0.4
)

pio.renderers.default = "notebook_connected"
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace3, secondary_y=True)
fig.add_trace(trace1)
fig.add_trace(trace2)
fig['layout'].update(height = 600, width = 800, title = 'Bonsoir')
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))

add = 0.1
y_max_quotes = max(-df_final_.neg.min(), df_final_.pos.max()) + add
y_max_stock = max(-stock_analysis.stock_price.min(), stock_analysis.stock_price.max()) + add

fig.update_yaxes(range=[-y_max_quotes, y_max_quotes], secondary_y=False)
fig.update_yaxes(range=[-y_max_stock, y_max_stock], secondary_y=True)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.update_layout(bargap=0.1, bargroupgap = 0, template='ggplot2')
iplot(fig)

In [ ]:
fig = px.bar(df_final_, x='date', y='score_pos')
fig.update_layout(
title={
        'text' : 'Distribution of the positive quotes according to time',
        'x':0.5,
        'xanchor': 'center'},
xaxis_title_text='date', # xaxis label
yaxis_title_text='frequency of positive quotes', # yaxis label
bargap=0.2, # gap between bars of adjacent location coordinates
bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))

fig.update_layout(bargap=0.1,
                bargroupgap = 0,
                )
fig.show()

# Graphics

***

In [ ]:
# separate the positive and negative quotes
pos_quotes = fuck[fuck['sentiment'] == 'Positive']
neg_quotes = fuck[fuck['sentiment'] == 'Negative']
neut_quotes = fuck[fuck['sentiment'] == 'Neutral']

In [ ]:
# plot the distribution of the neutral quotes according to time
neut_per_day = pd.DataFrame(neut_quotes.groupby(neut_quotes.date.dt.date).count()['sentiment'])
neut_per_day.index.rename('Date')
neut_per_day.reset_index(inplace=True)

# plot the distribution of the positive quotes according to time
pos_per_day = pd.DataFrame(pos_quotes.groupby(pos_quotes.date.dt.date).count()['sentiment'])
pos_per_day.index.rename('Date')
pos_per_day.reset_index(inplace=True)

# plot the distribution of the negative quotes according to time
neg_per_day = pd.DataFrame(neg_quotes.groupby(neg_quotes.date.dt.date).count()['sentiment'])
neg_per_day.index.rename('Date')
neg_per_day.reset_index(inplace=True)

In [ ]:
fig = px.bar(pos_per_day, x=pos_per_day['date'], y=pos_per_day['sentiment'])
fig.update_layout(
title={
        'text' : 'Distribution of the positive quotes according to time',
        'x':0.5,
        'xanchor': 'center'},
xaxis_title_text='date', # xaxis label
yaxis_title_text='frequency of positive quotes', # yaxis label
bargap=0.2, # gap between bars of adjacent location coordinates
bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))

fig.update_layout(bargap=0.1,
                bargroupgap = 0,
                )
fig.show()

In [ ]:
fig = px.bar(neg_per_day, x=neg_per_day['date'], y=neg_per_day['sentiment'])
fig.update_layout(
title={
        'text' : 'Distribution of the negative quotes according to time',
        'x':0.5,
        'xanchor': 'center'},
xaxis_title_text='date', # xaxis label
yaxis_title_text='frequency of negative quotes', # yaxis label
bargap=0.2, # gap between bars of adjacent location coordinates
bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))

fig.update_layout(bargap=0.1,
                bargroupgap = 0,
                )
fig.show()

In [ ]:
# Initialize figure
all_quotes_per_day = pos_per_day.merge(neg_per_day, how="right", on="date").merge(neut_per_day, how="left", on="date")
all_quotes_per_day.date = all_quotes_per_day.date.apply(lambda x : str(x))
all_quotes_per_day.rename({"sentiment_x": "Positive", "sentiment_y": "Negative", "sentiment":"Neutral"},axis=1,inplace=True)
all_quotes_per_day['All'] = all_quotes_per_day.Positive + all_quotes_per_day.Negative + all_quotes_per_day.Neutral


fig = go.Figure()
ymax = all_quotes_per_day.All.max()

# Add Traces

fig.add_trace(
    go.Bar(x=all_quotes_per_day.date,
            y=all_quotes_per_day.All,
            name="All",
            marker_color="blue"))

fig.add_trace(
    go.Bar(x=all_quotes_per_day.date,
            y=all_quotes_per_day.Negative,
            name="Negative",
                visible=False,
            marker_color="red"))

fig.add_trace(
    go.Bar(x=neut_per_day.date,
            y=all_quotes_per_day.Positive,
            name="Positive",
            visible = False,
            marker_color="green"))

fig.add_trace(
    go.Bar(x=neut_per_day.date,
            y=all_quotes_per_day.Neutral,
            name="Neutral",
                visible=False,
            marker_color="gray"))

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(label="All",
                    method="update",
                    args=[{"visible": [True, False, False, False]},
                        {"title": "All quotes",
                            "annotations": []}]),
                dict(label="Negative",
                    method="update",
                    args=[{"visible": [False, True, False, False]},
                        {"title": "Negative quotes"}]),
                dict(label="Positive",
                    method="update",
                    args=[{"visible": [False, False, True, False]},
                        {"title": "Positive quotes"}]),
                dict(label="Neutral",
                    method="update",
                    args=[{"visible": [False, False, False, True]},
                        {"title": "Neutral quotes"}]),
            ]),
        )
    ])

# Set title
fig.update_layout(
    title_text="All quotes",
    xaxis_domain=[0.05, 1.0],
    yaxis_range =[0,ymax],
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Frequency of quotes', # yaxis label
    bargap=0.1,
    bargroupgap = 0
)
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))
fig.show()

In [ ]:
from task3 import *
task3()

***

In [ ]:
display(speakers_labels.head(10))

In [ ]:
print(get_page_views_per_year('Shameel Joosub', 2015))

In [ ]:
yolo = filtered_quotes.copy()
yolo['year'] = filtered_quotes.date.apply(lambda date: date.year)
display(yolo)

In [ ]:
try_ = pd.DataFrame({'B': [0, 1, 2, 100, 4, 0, 1, 2, 100, 4]})
try_.rolling(window = 3).mean()

# Data selection for the model

***

In [ ]:
def func_pos(row):
    score = 0
    if row.sentiment > 0:
        score = row.absolute_score
    return score

def func_neg(row):
    score = 0
    if row.sentiment < 0:
        score = -row.absolute_score
    return score

In [ ]:
quotes_score = df_final.copy()

quotes_score['positive'] = quotes_score.progress_apply(func_pos, axis = 1)
quotes_score['negative'] = quotes_score.progress_apply(func_neg, axis = 1)

quotes_score = quotes_score[['date', 'positive', 'negative']].dropna().groupby(['date']).sum()

quotes_score['total'] = quotes_score.positive + quotes_score.positive

quotes_score = quotes_score.reset_index(drop = False)

display(quotes_score)

In [ ]:
stock_name = "AAPL"
year_start = 2015
year_end = 2020

apple_stock_market = yf.download(stock_name, start=f'{year_start}-01-01', end=f'{year_end}-12-31', progress = False)
apple_stock_market.reset_index(inplace=True)
date_max = '2020-04-16'
apple_stock_market = apple_stock_market[apple_stock_market.Date <= date_max]

apple_stock_market = apple_stock_market.copy()
apple_stock_market['change'] = apple_stock_market.Close - apple_stock_market.Open

apple_stock_market['date'] = apple_stock_market['Date']

apple_stock_market = apple_stock_market[apple_stock_market.columns.difference(['Date'])]

display(apple_stock_market)


In [ ]:
apple_stock_market.to_pickle('data/apple_stock_market.pkl')
quotes_score.to_pickle('data/quotes_score.pkl')